In [10]:
import cv2
import mediapipe as mp
import numpy as np
import math as m
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
cap = cv2.VideoCapture(0)
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

def calcdistance(x1, y1, x2, y2):
    dist = m.sqrt((x2-x1)**2+(y2-y1)**2)
    return dist

count= 0
stage= None
reps= int(input("How many Pushups are you going for?"))
reps_count=0
over= False
# Render detections
green = (0, 255, 0)
red = (0, 0, 255)

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.2, min_tracking_confidence=0.2) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            l_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            offset = calcdistance(l_hip[0], l_hip[1], r_hip[0], r_hip[1])

            # Calculate angle
            squat_angle = calculate_angle(l_hip, l_knee, l_ankle)
            # Visualize angle
            cv2.putText(image, str(squat_angle), 
                           tuple(np.multiply(l_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            #logic
           
            if squat_angle<100 and offset < 100 and not over:
                stage="Down"

            if squat_angle>110 and offset < 100 and stage=="Down" and not over:
                stage="Up"
                count +=1
            if count==reps_count:
                over=True  
            if offset < 100:
                cv2.putText(image, "Camera not aligned with body, turn to your sides", (400, 470),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA
                                )
            else:
                cv2.putText(image, "Camera Aligned Properly", (400, 470),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, green, 2, cv2.LINE_AA
                                )
        except:
            pass
        #Visualising result
        cv2.putText(image, "Count", (200, 470),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA
                                )
        cv2.putText(image, str(count), (400, 470),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA
                                )
        #offset logic
        
            
        
        status_color= green if squat_angle<100 and offset< 100 else red
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color= status_color, thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Squats', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

How many Pushups are you going for? 9
